# Automatic model tuning of TensorFlow models with SageMaker
This lab demonstrates the power of **Amazon SageMaker's automatic model tuning capability**, also known as hyperparameter optimization (HPO). Instead of a labor intensive process of trial and error that could take days or weeks, [automatic model tuning](https://docs.aws.amazon.com/sagemaker/latest/dg/automatic-model-tuning.html) let's a data scientist ask SageMaker to find the optimal set of hyperparameters. 

The notebook shows how to provide a set of parameters to tune and ranges to consider, a metric to optimize on, some limits on the number of jobs to consider and the compute capacity to leverage. A SageMaker tuning job then efficiently explores options using a Bayesian optimization. SageMaker creates a set of models and highlights which one is optimal given your constraints. The resulting model is ready for deployment behind an endpoint or for batch predictions.

## Setup
For this notebook, we simply get our security role and establish some parameters for use of S3.

In [1]:
import sagemaker
from sagemaker import get_execution_role
import boto3

client  = boto3.client(service_name='sagemaker')

role = get_execution_role()
print(role)
sess = sagemaker.Session()

bucket = sess.default_bucket() # or custom bucket name
prefix = 'DEMO-TF-image-classification-birds'
JOB_PREFIX = 'tf-hpo-ic'

arn:aws:iam::355151823911:role/service-role/AmazonSageMaker-ExecutionRole-20180515T132694


This notebook relies on execution of previous notebooks in this workshop. Specifically, it assumes the image data has been prepared and uploaded to s3. Here we just define exactly where the training jobs will pull the data from.

In [3]:
train_inputs = 's3://{}/{}/train'.format(bucket, prefix)
val_inputs   = 's3://{}/{}/validation'.format(bucket, prefix)
test_inputs  = 's3://{}/{}/test'.format(bucket, prefix)
print('Training data:   {}\nValidation data: {}\nTest data:       {}'.format(train_inputs, val_inputs, test_inputs))

Training data:   s3://sagemaker-us-east-1-355151823911/DEMO-TF-image-classification-birds/train
Validation data: s3://sagemaker-us-east-1-355151823911/DEMO-TF-image-classification-birds/validation
Test data:       s3://sagemaker-us-east-1-355151823911/DEMO-TF-image-classification-birds/test


Here are the classes that have been uploaded to s3 for training against.

In [4]:
!aws s3 ls $train_inputs/

                           PRE 013.Bobolink/
                           PRE 017.Cardinal/


## Create hyperparameter tuning job
To use Amazon SageMaker's automatic model tuning capability, you create a tuning job, which in turn will launch a set of SageMaker training jobs. As when creating a training job directly, you first establish a set of hyperparameters, some metric definitions, and then a TensorFlow estimator which will be fed a Python training script.

In [5]:
from sagemaker.tensorflow import TensorFlow
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner

In [6]:
hyperparameters = {'initial_epochs': 5, 'tuning_epochs': 35, 
                   'data_dir': '/opt/ml/input/data',
                   'dropout': 0.5, 'num_fully_connected_layers': 1}

metric_definitions=[{'Name' : 'validation:acc', 
                     'Regex': '.*step.* - val_acc: (.*$)'},
                    {'Name' : 'validation:loss', 
                     'Regex': '- val_loss: (.*?) '},
                    {'Name' : 'acc', 
                     'Regex': '.*step.* - acc: (.*?) '},
                    {'Name' : 'loss', 
                     'Regex': '.*step.* - loss: (.*?) '}]

estimator = TensorFlow(entry_point='train-mobilenet.py',
                    source_dir='code',
                    role=role,
                    framework_version='1.13.1',
                    train_instance_count=1,
                    train_instance_type='ml.p3.2xlarge',
                    hyperparameters=hyperparameters,
                    metric_definitions=metric_definitions,
                    py_version='py3',
                    base_job_name=JOB_PREFIX)

More interestingly, here is the part that is unique to creating the tuning job. You define a set of hyperparameter ranges that you want SageMaker to explore via training jobs. For our example, we focus on the number of [fine tuning](https://www.pyimagesearch.com/2019/06/03/fine-tuning-with-keras-and-deep-learning/) epochs, the dropout ratio, and the fine tuning learning rate. If we were to figure this out manually, it would take significant time and lots of trial and error. With SageMaker, we can hand off that job and find the optimal settings. 

In [30]:
hyperparameter_ranges = {'fine_tuning_epochs': IntegerParameter(35, 55),
                         'dropout': ContinuousParameter(0.2, 0.7),
                         'fine_tuning_lr': ContinuousParameter(0.00001, 0.001)}

In [31]:
objective_metric_name = 'validation:acc'
objective_type = 'Maximize'

In [32]:
tuner = HyperparameterTuner(estimator,
                            objective_metric_name,
                            hyperparameter_ranges,
                            metric_definitions,
                            max_jobs=8,
                            max_parallel_jobs=2,
                            objective_type=objective_type,
                           base_tuning_job_name=JOB_PREFIX)

In [33]:
inputs = {'train':train_inputs, 'test': test_inputs, 'validation': val_inputs}

With the tuning job established, we can now launch the job and then check back to see what parameters are best suited to our image classifier.

In [34]:
tuner.fit(inputs)

In [35]:
status = boto3.client('sagemaker').describe_hyper_parameter_tuning_job(
    HyperParameterTuningJobName=tuner.latest_tuning_job.job_name)['HyperParameterTuningJobStatus']
print('Tuning job: {}, Status: {}'.format(tuner.latest_tuning_job.job_name, status))

Tuning job: tf-hpo-ic-190908-1357, Status: InProgress


## Analyze tuning job results
In the remainder of this notebook, we perform some analysis on the results of the tuning job. This helps us gain insight into which parameters were most influential and may also help generate ideas for other tuning jobs that would help get even closer to our objective. The SageMaker console also provides a good way to track the job and review results.

In [36]:
tuning_job_name = tuner.latest_tuning_job.job_name

Here we can monitor the progress of the overall tuning job, finding out how many jobs are completed.

In [37]:
tuning_job_result = client.describe_hyper_parameter_tuning_job(HyperParameterTuningJobName=tuning_job_name)

status = tuning_job_result['HyperParameterTuningJobStatus']
if status != 'Completed':
    print('Reminder: the tuning job has not been completed.')
    
job_count = tuning_job_result['TrainingJobStatusCounters']['Completed']
print("%d training jobs have completed" % job_count)
    
is_minimize = (tuning_job_result['HyperParameterTuningJobConfig']['HyperParameterTuningJobObjective']['Type'] != 'Maximize')
objective_name = tuning_job_result['HyperParameterTuningJobConfig']['HyperParameterTuningJobObjective']['MetricName']

7 training jobs have completed


Here we take a look at the parameters that were used for the best model produced thus far.

In [38]:
from pprint import pprint
if tuning_job_result.get('BestTrainingJob',None):
    print("Best model found so far:")
    pprint(tuning_job_result['BestTrainingJob'])
else:
    print("No training jobs have reported results yet.")

Best model found so far:
{'CreationTime': datetime.datetime(2019, 9, 8, 13, 57, 7, tzinfo=tzlocal()),
 'FinalHyperParameterTuningJobObjectiveMetric': {'MetricName': 'validation:acc',
                                                 'Value': 1.0},
 'ObjectiveStatus': 'Succeeded',
 'TrainingEndTime': datetime.datetime(2019, 9, 8, 14, 3, 40, tzinfo=tzlocal()),
 'TrainingJobArn': 'arn:aws:sagemaker:us-east-1:355151823911:training-job/tf-hpo-ic-190908-1357-001-04afcf0c',
 'TrainingJobName': 'tf-hpo-ic-190908-1357-001-04afcf0c',
 'TrainingJobStatus': 'Completed',
 'TrainingStartTime': datetime.datetime(2019, 9, 8, 13, 59, 1, tzinfo=tzlocal()),
 'TunedHyperParameters': {'dropout': '0.496287895302724',
                          'fine_tuning_epochs': '35',
                          'fine_tuning_lr': '6.235116341787422e-05'}}


Here we produce a grid view of all the jobs, their parameters, and their results. They are sorted in descending order of their final objective value (best metric at the top, worst at the bottom).

In [39]:
import pandas as pd

tuner_analytics = sagemaker.HyperparameterTuningJobAnalytics(tuning_job_name)

full_df = tuner_analytics.dataframe()

if len(full_df) > 0:
    df = full_df[full_df['FinalObjectiveValue'] > -float('inf')]
    if len(df) > 0:
        df = df.sort_values('FinalObjectiveValue', ascending=is_minimize)
        print("Number of training jobs with valid objective: %d" % len(df))
        print({"lowest":min(df['FinalObjectiveValue']),"highest": max(df['FinalObjectiveValue'])})
        pd.set_option('display.max_colwidth', -1)  # Don't truncate TrainingJobName        
    else:
        print("No training jobs have reported valid results yet.")
        
df

Number of training jobs with valid objective: 7
{'lowest': 0.478300005197525, 'highest': 1.0}


,FinalObjectiveValue,TrainingElapsedTimeSeconds,TrainingEndTime,TrainingJobName,TrainingJobStatus,TrainingStartTime,dropout,fine_tuning_epochs,fine_tuning_lr
1,1.0000,250.0,2019-09-08 14:22:51+00:00,tf-hpo-ic-190908-1357-007-2bd73a98,Completed,2019-09-08 14:18:41+00:00,0.431433,53.0,0.000010
2,1.0000,235.0,2019-09-08 14:19:44+00:00,tf-hpo-ic-190908-1357-006-65c428c8,Completed,2019-09-08 14:15:49+00:00,0.296764,40.0,0.000105
5,1.0000,223.0,2019-09-08 14:10:18+00:00,tf-hpo-ic-190908-1357-003-e88ecaec,Completed,2019-09-08 14:06:35+00:00,0.455496,35.0,0.000602
7,1.0000,279.0,2019-09-08 14:03:40+00:00,tf-hpo-ic-190908-1357-001-04afcf0c,Completed,2019-09-08 13:59:01+00:00,0.496288,35.0,0.000062
6,0.9565,275.0,2019-09-08 14:03:58+00:00,tf-hpo-ic-190908-1357-002-7e01f400,Completed,2019-09-08 13:59:23+00:00,0.346427,47.0,0.000026
0,0.5217,206.0,2019-09-08 14:25:22+00:00,tf-hpo-ic-190908-1357-008-1e99183d,Completed,2019-09-08 14:21:56+00:00,0.332022,35.0,0.000011
3,0.4783,224.0,2019-09-08 14:16:33+00:00,tf-hpo-ic-190908-1357-005-08170720,Completed,2019-09-08 14:12:49+00:00,0.291764,40.0,0.000109


With the following chart, we can see how well SageMaker's Bayesian optimization was able to explore the search space of possible hyperparameters over time. In our case, we only ran a few jobs with a few hyperparameter ranges. For a production tuning job with many more jobs and parameter ranges, this chart is more compelling.

In [40]:
import bokeh
import bokeh.io
bokeh.io.output_notebook()
from bokeh.plotting import figure, show
from bokeh.models import HoverTool

class HoverHelper():

    def __init__(self, tuning_analytics):
        self.tuner = tuning_analytics

    def hovertool(self):
        tooltips = [
            ("FinalObjectiveValue", "@FinalObjectiveValue"),
            ("TrainingJobName", "@TrainingJobName"),
        ]
        for k in self.tuner.tuning_ranges.keys():
            tooltips.append( (k, "@{%s}" % k) )

        ht = HoverTool(tooltips=tooltips)
        return ht

    def tools(self, standard_tools='pan,crosshair,wheel_zoom,zoom_in,zoom_out,undo,reset'):
        return [self.hovertool(), standard_tools]

hover = HoverHelper(tuner_analytics)

p = figure(plot_width=900, plot_height=400, tools=hover.tools(), x_axis_type='datetime')
p.circle(source=df, x='TrainingStartTime', y='FinalObjectiveValue')
show(p)

Loading BokehJS ...

Lastly, we take a look at how significantly each of our hyperparameters impacted the final objective value.

In [41]:
ranges = tuner_analytics.tuning_ranges
figures = []
for hp_name, hp_range in ranges.items():
    categorical_args = {}
    if hp_range.get('Values'):
        # This is marked as categorical.  Check if all options are actually numbers.
        def is_num(x):
            try:
                float(x)
                return 1
            except:
                return 0           
        vals = hp_range['Values']
        if sum([is_num(x) for x in vals]) == len(vals):
            # Bokeh has issues plotting a "categorical" range that's actually numeric, so plot as numeric
            print("Hyperparameter %s is tuned as categorical, but all values are numeric" % hp_name)
        else:
            # Set up extra options for plotting categoricals.  A bit tricky when they're actually numbers.
            categorical_args['x_range'] = vals

    # Now plot it
    p = figure(plot_width=500, plot_height=500, 
               title="Objective vs %s" % hp_name,
               tools=hover.tools(),
               x_axis_label=hp_name, y_axis_label=objective_name,
               **categorical_args)
    p.circle(source=df, x=hp_name, y='FinalObjectiveValue')
    figures.append(p)
show(bokeh.layouts.Column(*figures))